# Projeto 1 - Ciência dos Dados

Nome: Carlos Eduardo Porciuncula Yamada

Nome: Pedro Henrique de Sousa da Silva

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Obtenção de tweets

## Preparando o ambiente no jupyter:

In [7]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [8]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@projetocdados1***

In [9]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @projetocdados1

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas para construção da base de dados:

### Escolha de um produto e coleta das mensagens


In [10]:
#Produto escolhido:
produto = 'Apple'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [15]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    
    try:
        #Tenta acessar atributo 'retweeted_status'
        # Se mensagem não possuir este atributo dá erro
        # e vai para a clausula "except"
        msg.retweeted_status.full_text
    except AttributeError:  
        #Entra aqui sempre que tweet não for retweet
        msgs.append(msg.full_text.lower())

    
    i += 1
    
    temporario=list(set(list(msgs)))
    if len(temporario) >= n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)
len(msgs)


KeyboardInterrupt: 

In [16]:
msgs=list(set(list(msgs)))

Salvando os dados em uma planilha Excel:

In [17]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

**Importante: Caso classifique um percentual pequeno de tweets relevantes ou de não relevantes, deve voltar a este notebook e coletar mais tweets diferentes do produto escolhido pelo grupo.**

In [19]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Paula Yamada\OneDrive\Área de Trabalho\2021.2\cdados\cdados-projeto-1


In [60]:
dft = pd.read_excel('Apple.xlsx', sheet_name='Treinamento')

In [61]:
dft

,Treinamento,Classificação
0,e já temos uma hashflag ativada para o evento ...,1.0
1,o amo tanto https://t.co/8q8kwbchxg,NaN
2,apple ta a oferecer airpods na compra de um ma...,NaN
3,"primeiro eu e a larissa compra o airpods pro, ...",NaN
4,@autcoliveira eu tô na minha cabeça você é a a...,NaN
...,...,...
295,"como o chapa @marketto007 avisou, a capa da se...",1.0
296,essa musica cheira a vodka kkkkkkk https://t.c...,0.0
297,"caras, me mimei com um macbook pro e não consi...",1.0
298,@estadao se o conforto não o atrai então que m...,1.0


In [58]:
dfc = pd.read_excel('Apple.xlsx', sheet_name='Teste')

In [70]:
dfc

,Teste,Classificação
0,apple music horrível dou play e a música não t...,1
1,brilha brilha https://t.co/qdluo54yjn,0
2,muitos sonham em ter essa na discografia mas s...,0
3,vocals https://t.co/knkwca4vwn,0
4,#apple #iphone - bloquez vos agendas 🧐😎 https:...,1
...,...,...
195,"esse apple pay é um adianto, não preciso mais ...",1
196,2:25 meu deus https://t.co/ybl9vck3j9,0
197,a apple mudou o endereço da entrega por algum ...,1
198,@dudaslz_ @vmoronicius amiga para a menina lá ...,1


In [59]:
dfc

,Teste,Classificação
0,apple music horrível dou play e a música não t...,1
1,brilha brilha https://t.co/qdluo54yjn,0
2,muitos sonham em ter essa na discografia mas s...,0
3,vocals https://t.co/knkwca4vwn,0
4,#apple #iphone - bloquez vos agendas 🧐😎 https:...,1
...,...,...
195,"esse apple pay é um adianto, não preciso mais ...",1
196,2:25 meu deus https://t.co/ybl9vck3j9,0
197,a apple mudou o endereço da entrega por algum ...,1
198,@dudaslz_ @vmoronicius amiga para a menina lá ...,1


In [107]:
# ABA 'Treinamento' DA PLANILHA 'Apple.xlsx'
tbin_r = dft.isin([1])

tfr = tbin_r['Classificação'] == True
tfi = tbin_r['Classificação'] == False

tdfr = dft.loc[tfr, :]
tdfi = dft.loc[tfi, :]

In [108]:
# ABA 'Teste' DA PLANILHA 'Apple.xlsx'
cbin_r = dfc.isin([1])
cfr = cbin_r['Classificação'] == True
cfi = cbin_r['Classificação'] == False

cdfr = dfc.loc[cfr, :]
cdfi = dfc.loc[cfi, :]